In [ ]:
%%html
<!-- 
If you can see this code, this cell's output is not trusted.
Please execute this cell and save the notebook, or click File -> Trust Notebook
-->
<script>
var shown = true;

function filter_cells_by_tag(tag) {
    out = Array();
    all_cells = Jupyter.notebook.get_cells()
    for (var i=0; i<all_cells.length; i++) {
        var curr_cell = all_cells[i];
        var tags = curr_cell._metadata.tags;
        if (tags != undefined) {
            for (var j=0; j<tags.length; j++) {
                var curr_tag = tags[j];
                if (curr_tag == tag) {
                    out.push(curr_cell);
                    break;
                }
            }
        }
    }
    return out;
}

function set_cell_visibility(tag, show, input_only) {
    var cells = Jupyter.notebook.get_cells();
    var marked_cells = filter_cells_by_tag(tag);
    for (var i=0; i<marked_cells.length; i++) {
        var curr_cell = marked_cells[i];
        if (input_only) {
            obj = curr_cell.input
        } else {
            obj = curr_cell.element
        }
        if (show) {
            obj.show();
        } else {
            obj.hide();
        }
    }
}

function toggle_cell_visibility(tag) {
    set_cell_visibility(tag, shown, false)
    shown = ! shown;
}

set_cell_visibility('execution_cell', false, true);
</script>
To toggle visibility of explanation cells click <a href="javascript:toggle_cell_visibility('explanatory_cell')">here</a>


# SEC Filing Section Pipeline

This notebook defines the pipeline for extracting the narrative text sections
from the 10-K, 10-Q, and S-1 filings. This notebook contains both
exploration code and the code for defining the API. Code cells marked
with `#pipeline-api` are included in the API definition.

To demonstrate how off-the-shelf Unstructured Bricks extract
meaningful data from complex source documents, we will apply
a series of Bricks with explanations before defining the API.

#### Table of Contents

1. [Pulling in Raw Documents](#raw)
1. [Reading the Document](#reading)
1. [Custom Partitioning Bricks](#custom)
1. [Cleaning Bricks](#cleaning)
1. [Staging Bricks](#staging)
1. [Define the Pipeline API](#pipeline)

## Section 1: Pulling in Raw Documents <a id="raw"></a>

First, let's pull in a filing from the SEC EDGAR database.
In this case, we'll pull in the most recent 10-K for Royal Gold (RGLD),
a publicly traded precious metals company.

In [ ]:
from prepline_sec_filings.fetch import (
    get_form_by_ticker, open_form_by_ticker, get_filing
)

# Fetch the 2021 10-K instead of the most recent one
rgld_cik = 85535
accession_number = "000155837021011343"
text = get_filing(rgld_cik,
                  accession_number,
                  company='Unstructured Technologies', 
                  email='support@unstructured.io')

# Alternatively, use the following to fetch the most recent filing:
# text = get_form_by_ticker(
#    'rgld', 
#    '10-K', 
#    company='Unstructured Technologies', 
#    email='support@unstructured.io'
#)


In [ ]:
print(text[1375:3284])

<XBRL>
<?xml version='1.0' encoding='UTF-8'?>

      <!-- iXBRL document created with: Toppan Merrill Bridge iXBRL 9.6.7811.37134 -->
      <!-- Based on: iXBRL 1.1 -->
      <!-- Created on: 8/11/2021 10:45:07 PM -->
      <!-- iXBRL Library version: 1.0.7811.37150 -->
      <!-- iXBRL Service Job ID: 19f8db26-9ac2-4427-9c71-ed8734c57db7 -->

  <html xmlns:us-gaap="http://fasb.org/us-gaap/2020-01-31" xmlns:link="http://www.xbrl.org/2003/linkbase" xmlns:country="http://xbrl.sec.gov/country/2020-01-31" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:rgld="http://www.royalgold.com/20210630" xmlns:xbrldt="http://xbrl.org/2005/xbrldt" xmlns:ixt-sec="http://www.sec.gov/inlineXBRL/transformation/2015-08-31" xmlns:srt="http://fasb.org/srt/2020-01-31" xmlns:ix="http://www.xbrl.org/2013/inlineXBRL" xmlns:ref="http://www.xbrl.org/2006/ref" xmlns:utr="http://www.xbrl.org/2009/utr" xmlns:xbrli="http://www.xbrl.org/2003/instance" xmlns:dei="http://xbrl.sec.gov/dei/2020-01-31" xmlns:iso4

Wow! We're able to pull in the document, but it's really messy. 
To help, we'll apply Unstructured Bricks to extract the information we're most interested in. Ultimately, we want to be able to ask the API
for a section and get back the narrative text within that section. Once
we extract the narrative text, we can spin up a labeling task or send
it to a downstream ML service for inference.

```json
[
  {
    "text": "You should carefully consider the risks described in this section. Our future performance is subject to risks and uncertainties that could have a material adverse effect on our business, results of operations, and financial condition and the trading price of our common stock. We may be subject to other risks and uncertainties not presently known to us. In addition, please see our note about forward-looking statements included in the MD&A.",
    "type": "NarrativeText"
  },
  {
    "text": "Our revenue is subject to volatility in metal prices, which could negatively affect our results of operations or cash flow.",
    "type": "NarrativeText"
  },
  {
    "text": "Market prices for gold, silver, copper, nickel, and other metals may fluctuate widely over time and are affected by numerous factors beyond our control. These factors include metal supply and demand, industrial and jewelry fabrication, investment demand, central banking actions, inflation expectations, currency values, interest rates, forward sales by metal producers, and political, trade, economic, or banking conditions.",
    "type": "NarrativeText"
  },
```

## Section 2: Reading the Document <a id="reading"></a>

The first step is to identify and categorize text elements within the
document. When we get the SEC document from EDGAR, it is in `.xml` format. Like most text documents, XML documents contains text data we're
interested along with other information that we'd like to discard. In
an XML or HTML document, that could be styling or formatting tags.
Other document types, like PDFs, might have headers, footers, and page
numbers we're not interested in.

To help with this, Unstructured has created a library of 
***partitioning bricks*** to break down a document into coherent chunks.
Different document types have different partitioning methods. For HTML
or XML documents, we identify sections based on text tags and use
NLP to differentiate between titles, narrative text, and other section
types. For PDF documents, we use visual clues such as the layout of the
document.

In [ ]:
from unstructured.documents.html import HTMLDocument

html_document = HTMLDocument.from_string(text).doc_after_cleaners(skip_headers_and_footers=True, skip_table_text=True)

Here, we see that the `HTMLDocument` Brick was able to extract
text from the raw source XML document. This is a generic
`HTMLDocument` class that does not have any special knowledge about
the structure of EDGAR documents.

In [ ]:
for element in html_document.pages[0].elements[71:75]:
    print(element)
    print("\n")

Acquisition and Management of Stream Interests—A metal stream is a purchase agreement that provides, in exchange for an upfront deposit payment, the right to purchase all or a portion of one or more metals produced from a mine, at a price determined for the life of the transaction by the purchase agreement. As of June 30, 2021, we owned eight stream interests, which are on six producing properties and two development stage properties. Our stream interests accounted for approximately 69% and 72% of our total revenue for each of the fiscal years ended June 30, 2021, and 2020, respectively. We expect stream interests to represent a significant portion of our total revenue.


Acquisition and Management of Royalty Interests—Royalties are non-operating interests in mining projects that provide the right to a percentage of revenue or metals produced from the project after deducting specified costs, if any. As of June 30, 2021, we owned royalty interests on 35 producing properties, 15 developm

Internally, the sections of text are represented using `Title` and
`NarrativeText` classes, as shown below. Bricks that are executed while
processing the document distinguish titles from narrative text.

In [ ]:
html_document.pages[0].elements[71:75]

[<unstructured.documents.html.HTMLListItem>,
 <unstructured.documents.html.HTMLListItem>,
 <unstructured.documents.html.HTMLTitle>,
 <unstructured.documents.html.HTMLNarrativeText>]

Below, we show Bricks identifying possible titles and narrative text
sections. `"Regulation"` is identified as a title and not narrative text
because it is short and does not contain any verbs. Conversely, longer
sections with multiple, complex sentences are identified as narrative.

In [ ]:
from unstructured.nlp.partition import is_possible_title

is_possible_title("Regulation")

True

In [ ]:
is_possible_title("""Operators of the mines that are subject to our 
stream and royalty interests must comply with numerous environmental, 
mine safety, land use, waste disposal, remediation and public health 
laws and regulations promulgated by federal, state, provincial and 
local governments in the United States, Canada, Chile, the Dominican 
Republic, Ghana, Mexico, Botswana, Australia and other countries where 
we hold interests. Although we, as a stream or royalty interest owner, 
are not""")

False

In [ ]:
from unstructured.nlp.partition import is_possible_narrative_text

is_possible_narrative_text("Regulation")

False

In [ ]:
is_possible_narrative_text("""Operators of the mines that are subject to our 
stream and royalty interests must comply with numerous environmental, 
mine safety, land use, waste disposal, remediation and public health 
laws and regulations promulgated by federal, state, provincial and 
local governments in the United States, Canada, Chile, the Dominican 
Republic, Ghana, Mexico, Botswana, Australia and other countries where 
we hold interests. Although we, as a stream or royalty interest owner, 
are not""")

True

## Section 3: Custom Partitioning Bricks <a id="custom"></a>

In addition to the partitioning bricks Unstructured provides out of the
box, a given recipe may require custom partitioning bricks. In this case,
we're interested in identifying sections within the SEC filing. To 
support that, we'll create a custom partitioning brick to identify 
section titles. This will help distinguish section headings from sub-headings.

In [ ]:
import re
from unstructured.documents.elements import Title

In [ ]:
ITEM_TITLE_RE = re.compile(
    r"(?i)item \d{1,3}(?:[a-z]|\([a-z]\))?(?:\.)?(?::)?"
)

In [ ]:
def is_10k_item_title(title: str) -> bool:
    """Determines if a title corresponds to a 10-K item heading."""
    return ITEM_TITLE_RE.match(title) is not None

This is looking pretty good! Those all look like the section headings
we're looking for. But wait, it seems like a few might be missing? It
turns out some of the titles contain extra whitespace characters. As a
result, they are not captured by the regex. Fortunately, Unstructured
has tools to help with that, which we'll cover in the next section.

In [ ]:
for element in html_document.elements:
    if is_possible_title(element.text, title_max_word_length=20) and is_10k_item_title(element.text):
        print(element)

ITEM 1A. RISK FACTORS


ITEM 16.    FORM 10-K SUMMARY


## Section 4: Cleaning Bricks <a id="cleaning"></a>

In addition to partitioning bricks, the Unstructured library has
***cleaning*** bricks for removing unwanted content from text. In this
case, we'll solve our whitespace problem by using the 
`clean_extra_whitespace` brick. Other uses for cleaning bricks include
cleaning out boilerplate, sentence fragments, and other segments
of text that could impact labeling tasks or the accuracy of
machine learning models. As with partitioning bricks, users can
include custom cleaning bricks in a pipeline.

In [ ]:
from unstructured.cleaners.core import clean_extra_whitespace

In [ ]:
for element in html_document.elements:
    element.text = clean_extra_whitespace(element.text)
    if is_possible_title(element.text, title_max_word_length=20) and is_10k_item_title(element.text):
        print(element)

ITEM 1. BUSINESS
ITEM 1A. RISK FACTORS


ITEM 1B. UNRESOLVED STAFF COMMENTS
ITEM 2. PROPERTIES


ITEM 3. LEGAL PROCEEDINGS
ITEM 4. MINE SAFETY DISCLOSURE
ITEM 5. MARKET FOR REGISTRANT’S COMMON EQUITY, RELATED STOCKHOLDER MATTERS AND ISSUER PURCHASES OF EQUITY SECURITIES
ITEM 6. RESERVED
ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS


ITEM 7A. QUANTITATIVE AND QUALITATIVE DISCLOSURE ABOUT MARKET RISK
ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA


ITEM 9. CHANGES IN AND DISAGREEMENTS WITH ACCOUNTANTS ON ACCOUNTING AND FINANCIAL DISCLOSURE
ITEM 9A. CONTROLS AND PROCEDURES
ITEM 9B. OTHER INFORMATION
ITEM 10. DIRECTORS, EXECUTIVE OFFICERS AND CORPORATE GOVERNANCE
ITEM 11. EXECUTIVE COMPENSATION
ITEM 12. SECURITY OWNERSHIP OF CERTAIN BENEFICIAL OWNERS AND MANAGEMENT AND RELATED STOCKHOLDER MATTERS
ITEM 13. CERTAIN RELATIONSHIPS AND RELATED TRANSACTIONS AND DIRECTOR INDEPENDENCE
ITEM 14. PRINCIPAL ACCOUNTANT FEES AND SERVICES
ITEM 15. EXHIBITS AND FINANCIAL STATEMENT SCHEDULES
ITEM 16. FORM 10-K SUMMARY


After applying the cleaning brick, we have a good looking list of 
section titles! With our set of bricks in place, we have what we
need to identify the risk section. For convenience, we've packaged them
up in the `SECDocument` class in the Python module for the pipeline. Notice
we include the `# pipeline-api` comment to include the `SECDocument`
import in the pipeline definition. Applying the full pipeline, we're
able to cleanly extract the risk narrative portion of the SEC filing.
The function is parameterized so we're easily able to get other sections
as well.

In [ ]:
# pipeline-api
from prepline_sec_filings.sections import section_string_to_enum, validate_section_names, SECSection
from prepline_sec_filings.sec_document import SECDocument, REPORT_TYPES, VALID_FILING_TYPES

In [ ]:
sec_document = SECDocument.from_string(text)
risk_narrative = sec_document.get_section_narrative(SECSection.RISK_FACTORS)

In [ ]:
for element in risk_narrative[:3]:
    print(element)
    print("\n")

You should carefully consider the risks described in this section. Our future performance is subject to risks and uncertainties that could have a material adverse effect on our business, results of operations, and financial condition and the trading price of our common stock. We may be subject to other risks and uncertainties not presently known to us. In addition, please see our note about forward-looking statements included in the MD&A.


Our revenue is subject to volatility in metal prices, which could negatively affect our results of operations or cash flow.


Market prices for gold, silver, copper, nickel, and other metals may fluctuate widely over time and are affected by numerous factors beyond our control. These factors include metal supply and demand, industrial and jewelry fabrication, investment demand, central banking actions, inflation expectations, currency values, interest rates, forward sales by metal producers, and political, trade, economic, or banking conditions.




## Section 5: Staging Bricks <a id="staging"></a>

After we've extracted the information we want, the next step is to
prepare the data for downstream tasks. This can include preparation
to:

- Feed the data into a downstream ML model
- Convert the data into a labeling task
- Upload the data into a database

To help with this, Unstructured has created ***staging bricks***
that preparing extracted data for downstream tasks. In this case,
we'll show how to format the data so that it can be easily
uploaded into LabelStudio as a labeling task. Other future uses cases include:

- Preparing data for downstream inference by Primer, Co:here, or Vannevar
- Chunking text to fit into attention windows for Huggingface
- Uploading data to Palantir Foundry

In [ ]:
from unstructured.staging.label_studio import stage_for_label_studio

In [ ]:
label_studio_data = stage_for_label_studio(risk_narrative)
label_studio_data[:5]

[{'data': {'text': 'You should carefully consider the risks described in this section. Our future performance is subject to risks and uncertainties that could have a material adverse effect on our business, results of operations, and financial condition and the trading price of our common stock. We may be subject to other risks and uncertainties not presently known to us. In addition, please see our note about forward-looking statements included in the MD&A.',
   'ref_id': '7a912bb639b547404be4ceaf5d9083a9'}},
 {'data': {'text': 'Our revenue is subject to volatility in metal prices, which could negatively affect our results of operations or cash flow.',
   'ref_id': 'd4cc8e0e0c2b68ef69282c5250b721c9'}},
 {'data': {'text': 'Market prices for gold, silver, copper, nickel, and other metals may fluctuate widely over time and are affected by numerous factors beyond our control. These factors include metal supply and demand, industrial and jewelry fabrication, investment demand, central bank

## Section 6: Define the Pipeline API <a id="pipeline"></a>

Once we have everything ready to go, we can package up the pipeline API
using the `# pipeline-api` comment. Unstructured has tooling that will
convert this notebook directly into a REST API. By convention, the REST
API calls the `pipeline_api` function when the API is invoked.
Once the REST API is up and running, users can make an HTTP request
to parse new documents.

In [ ]:
# pipeline-api
from enum import Enum
import re
import signal

from unstructured.staging.base import convert_to_isd
from prepline_sec_filings.sections import (
    ALL_SECTIONS,
    SECTIONS_10K,
    SECTIONS_10Q,
    SECTIONS_S1,
)

In [ ]:
# pipeline-api
class timeout:
    def __init__(self, seconds=1, error_message='Timeout'):
        self.seconds = seconds
        self.error_message = error_message
    def handle_timeout(self, signum, frame):
        raise TimeoutError(self.error_message)
    def __enter__(self):
        try:
            signal.signal(signal.SIGALRM, self.handle_timeout)
            signal.alarm(self.seconds)
        except ValueError:
            pass
    def __exit__(self, type, value, traceback):
        try:
            signal.alarm(0)
        except ValueError:
            pass

In [ ]:
# pipeline-api
def get_regex_enum(section_regex):
    class CustomSECSection(Enum):
        CUSTOM = re.compile(section_regex)
        
        @property
        def pattern(self):
            return self.value
            
    return CustomSECSection.CUSTOM

In [ ]:
# pipeline-api
import io
import csv
from typing import Dict
from unstructured.documents.elements import Text, NarrativeText, Title, ListItem
def convert_to_isd_csv(results:dict) -> str:
    """
    Returns the representation of document elements as an Initial Structured Document (ISD)
    in CSV Format.
    """
    csv_fieldnames: List[str] = ["section", "element_type", "text"]
    new_rows = []
    for section, section_narrative in results.items():
        rows: List[Dict[str, str]] = convert_to_isd(section_narrative)
        for row in rows:
            new_row_item = dict()
            new_row_item["section"] = section
            new_row_item["element_type"] = row["type"]
            new_row_item["text"] = row["text"]
            new_rows.append(new_row_item)
        
    with io.StringIO() as buffer:
        csv_writer = csv.DictWriter(buffer, fieldnames=csv_fieldnames)
        csv_writer.writeheader()
        csv_writer.writerows(new_rows)
        return buffer.getvalue()

In [ ]:
# pipeline-api
from unstructured.staging.label_studio import stage_for_label_studio

# List of valid response schemas
LABELSTUDIO = "labelstudio"
ISD = "isd"

def pipeline_api(text, response_type="application/json", response_schema="isd", m_section=[], m_section_regex=[]):
    """Many supported sections including: RISK_FACTORS, MANAGEMENT_DISCUSSION, and many more"""
    validate_section_names(m_section)
    
    sec_document = SECDocument.from_string(text)
    if sec_document.filing_type not in VALID_FILING_TYPES:
        raise ValueError(
            f"SEC document filing type {sec_document.filing_type} is not supported, "
            f"must be one of {','.join(VALID_FILING_TYPES)}"
        )
    results = {}
    if m_section == [ALL_SECTIONS]:
        filing_type = sec_document.filing_type
        if filing_type in REPORT_TYPES:
            if filing_type.startswith("10-K"):
                m_section = [enum.name for enum in SECTIONS_10K]
            elif filing_type.startswith("10-Q"):
                m_section = [enum.name for enum in SECTIONS_10Q]
            else:
                raise ValueError(f"Invalid report type: {filing_type}")

        else:
            m_section = [enum.name for enum in SECTIONS_S1]
    for section in m_section:
        results[section] = sec_document.get_section_narrative(
            section_string_to_enum[section]
        )
    for i, section_regex in enumerate(m_section_regex):
        regex_enum = get_regex_enum(section_regex)
        with timeout(seconds=5):
            section_elements = sec_document.get_section_narrative(regex_enum)
            results[f"REGEX_{i}"] = section_elements
    if response_type == "application/json":
        if response_schema == LABELSTUDIO:
            return {section:stage_for_label_studio(section_narrative) for section, section_narrative in results.items()}
        elif response_schema == ISD:
            return {section:convert_to_isd(section_narrative) for section, section_narrative in results.items()}
        else:
            raise ValueError(f"output_schema '{response_schema}' is not supported for {response_type}")
    elif response_type == "text/csv":
        if response_schema != ISD:
            raise ValueError(f"output_schema '{response_schema}' is not supported for {response_type}")
        return convert_to_isd_csv(results)
    else:
        raise ValueError(f"response_type '{response_type}' is not supported")

As we can see, the output now matches what we wanted at the beginning
of the notebook.

In [ ]:
risk_narrative = pipeline_api(text, m_section=["RISK_FACTORS"])["RISK_FACTORS"]
risk_narrative[:5]

[{'element_id': '7a912bb639b547404be4ceaf5d9083a9',
  'coordinates': None,
  'text': 'You should carefully consider the risks described in this section. Our future performance is subject to risks and uncertainties that could have a material adverse effect on our business, results of operations, and financial condition and the trading price of our common stock. We may be subject to other risks and uncertainties not presently known to us. In addition, please see our note about forward-looking statements included in the MD&A.',
  'type': 'NarrativeText',
  'metadata': {}},
 {'element_id': 'd4cc8e0e0c2b68ef69282c5250b721c9',
  'coordinates': None,
  'text': 'Our revenue is subject to volatility in metal prices, which could negatively affect our results of operations or cash flow.',
  'type': 'NarrativeText',
  'metadata': {}},
 {'element_id': '96515fdbe9e0c547a557c2fe30137176',
  'coordinates': None,
  'text': 'Market prices for gold, silver, copper, nickel, and other metals may fluctuate 

In [ ]:
all_narratives = pipeline_api(text, m_section=["_ALL"])
for section, elems in all_narratives.items():
    print(section)
    print(elems[:4])
    print("---------------")

BUSINESS
[]
---------------
RISK_FACTORS
[{'element_id': '7a912bb639b547404be4ceaf5d9083a9', 'coordinates': None, 'text': 'You should carefully consider the risks described in this section. Our future performance is subject to risks and uncertainties that could have a material adverse effect on our business, results of operations, and financial condition and the trading price of our common stock. We may be subject to other risks and uncertainties not presently known to us. In addition, please see our note about forward-looking statements included in the MD&A.', 'type': 'NarrativeText', 'metadata': {}}, {'element_id': 'd4cc8e0e0c2b68ef69282c5250b721c9', 'coordinates': None, 'text': 'Our revenue is subject to volatility in metal prices, which could negatively affect our results of operations or cash flow.', 'type': 'NarrativeText', 'metadata': {}}, {'element_id': '96515fdbe9e0c547a557c2fe30137176', 'coordinates': None, 'text': 'Market prices for gold, silver, copper, nickel, and other me

In [ ]:
risk_narrative = pipeline_api(text, response_type="text/csv", m_section=["RISK_FACTORS"])

with open('risk_narrative.csv', 'w') as out:
    out.write(risk_narrative)

In [ ]:
all_narratives = pipeline_api(text, response_type="text/csv", m_section=["_ALL"])

with open('all_narratives.csv', 'w') as out:
    out.write(all_narratives)